In [1]:
!pip install wikipedia-api


In [2]:
!pip install firebase-admin

In [6]:
import pandas as pd
import requests

# Replace with your API keys
PEXELS_API_KEY = '4TVOWrMhOXc7KxDYD48wfrSMbDWty7kor11j5D7rB0Pae5TWT8CwQ9jK'
PIXABAY_API_KEY = '46982072-7531fb5ffc0361aecc8d872c7'
UNSPLASH_ACCESS_KEY = 'vTnfvmrRosNzqGHUFD4MJP1l_adh_b8bap1x9l0rM6A'

# Function to fetch image URL from Wikimedia Commons
def fetch_image_url_from_wikimedia(location):
    user_agent = "stg/1.0 (akhnasj1612@gmail.com)"  # Replace with your app's name or email
    headers = {'User-Agent': user_agent}

    search_url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&titles={location}&prop=images"
    response = requests.get(search_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch images for {location}. Status code: {response.status_code}")
        return None
    
    data = response.json()
    page_id = list(data['query']['pages'].keys())[0]
    if page_id == "-1":
        print(f"Page for {location} not found.")
        return None
    
    images = data['query']['pages'][page_id].get('images', [])
    if not images:
        print(f"No images found for {location}.")
        return None
    
    image_title = images[0]['title']
    image_url = f"https://commons.wikimedia.org/wiki/Special:FilePath/{image_title.replace(' ', '_')}"
    print(f"Image found for {location} on Wikimedia: {image_url}")
    return image_url

# Function to fetch image URL from Unsplash
def fetch_image_url_from_unsplash(location):
    search_url = f"https://api.unsplash.com/search/photos?query={location}&client_id={UNSPLASH_ACCESS_KEY}"
    response = requests.get(search_url)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch images from Unsplash for {location}.")
        return None
    
    data = response.json()
    if data['results']:
        image_url = data['results'][0]['urls']['regular']
        print(f"Image found for {location} on Unsplash: {image_url}")
        return image_url
    return None

# Function to fetch image URL from Pexels
def fetch_image_url_from_pexels(location):
    headers = {'Authorization': PEXELS_API_KEY}
    search_url = f"https://api.pexels.com/v1/search?query={location}&per_page=1"
    response = requests.get(search_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch images from Pexels for {location}.")
        return None
    
    data = response.json()
    if data['photos']:
        image_url = data['photos'][0]['src']['medium']
        print(f"Image found for {location} on Pexels: {image_url}")
        return image_url
    return None

# Function to fetch image URL from Pixabay
def fetch_image_url_from_pixabay(location):
    search_url = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={location}&image_type=photo"
    response = requests.get(search_url)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch images from Pixabay for {location}.")
        return None
    
    data = response.json()
    if data['hits']:
        image_url = data['hits'][0]['webformatURL']
        print(f"Image found for {location} on Pixabay: {image_url}")
        return image_url
    return None

# Function to find image URL by trying each source in order of preference
def find_image_url(location):
    image_url = fetch_image_url_from_wikimedia(location)
    if not image_url:
        image_url = fetch_image_url_from_unsplash(location)
    if not image_url:
        image_url = fetch_image_url_from_pexels(location)
    if not image_url:
        image_url = fetch_image_url_from_pixabay(location)
    return image_url or 'No Image Found'

# Load the locations CSV file
locations_df = pd.read_csv('C:\\Users\\AKHNA\\Desktop\\MCA\\Smart-Tour-Guide\\locations.csv', encoding='ISO-8859-1')

# Initialize an empty list to store the fetched image URLs
image_urls = []

# Loop through each location and fetch its image URL
for location in locations_df['Location']:  # Ensure 'Location' matches the column name in your CSV
    print(f"Fetching image for {location}...")
    image_url = find_image_url(location)
    image_urls.append(image_url)

# Add the image URLs as a new column to the dataframe
locations_df['Image_URL'] = image_urls

# Save the updated dataframe back to CSV
output_file = 'C:\\Users\\AKHNA\\Desktop\\MCA\\Smart-Tour-Guide\\locations_updated.csv'
try:
    locations_df.to_csv(output_file, index=False)
    print(f"CSV file successfully updated. The file is saved as {output_file}")
except Exception as e:
    print(f"Error writing CSV file: {e}")


Fetching image for India Gate...
Image found for India Gate on Wikimedia: https://commons.wikimedia.org/wiki/Special:FilePath/File:A_Mass_Run_against_Drug_Abuse,_on_the_occasion_of_the_International_Day_Against_Drug_Abuse_and_Illicit_Trafficking,_at_India_Gate,_in_New_Delhi_on_June_26,_2011.jpg
Fetching image for Humayun's Tomb...
Image found for Humayun's Tomb on Wikimedia: https://commons.wikimedia.org/wiki/Special:FilePath/File:"Capture_of_the_King_of_Delhi_by_Captain_Hodson".jpg
Fetching image for Akshardham Temple...
No images found for Akshardham Temple.
Image found for Akshardham Temple on Unsplash: https://images.unsplash.com/photo-1671137777317-877064fb882d?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2NzM1Njd8MHwxfHNlYXJjaHwxfHxBa3NoYXJkaGFtJTIwVGVtcGxlfGVufDB8fHx8MTczMTE0Njg5M3ww&ixlib=rb-4.0.3&q=80&w=1080
Fetching image for Waste to Wonder Park...
Page for Waste to Wonder Park not found.
Image found for Waste to Wonder Park on Unsplash: https://images.unsplash.com/photo-

In [16]:
# Updating firestore collection locations with URLs

import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase Admin SDK (ensure you have your Firebase credentials file)
cred = credentials.Certificate('C:\\Users\\AKHNA\\Downloads\\smart-tour-guide-3d30d-firebase-adminsdk-h17r2-bda949eabd.json')  # Update with your credentials path
firebase_admin.initialize_app(cred)

# Load the CSV with the image URLs
locations_df = pd.read_csv('C:\\Users\\AKHNA\\Desktop\\MCA\\Smart-Tour-Guide\\locations.csv', encoding='ISO-8859-1')

# Initialize Firestore client
db = firestore.client()

# Function to update Firestore 'locations' collection with image URLs
def update_locations_with_images():
    locations_ref = db.collection('locations')

    # Loop through each row in the DataFrame
    for _, row in locations_df.iterrows():
        location_name = row['Location']
        image_url = row['Image_URL']

        # Find the Firestore document by location name
        # Adjust this as per your unique identifier in Firestore, here we assume 'Location' is unique
        query = locations_ref.where('Location', '==', location_name).limit(1).stream()

        # Update the document with the new 'image_url' field
        for doc in query:
            doc_ref = locations_ref.document(doc.id)
            doc_ref.update({'image_url': image_url})
            print(f"Updated {location_name} with image URL: {image_url}")

# Run the function to update Firestore documents
update_locations_with_images()
print("Firestore 'locations' collection updated with image URLs.")


C:\Users\AKHNA\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


Updated India Gate with image URL: https://commons.wikimedia.org/wiki/Special:FilePath/File:A_Mass_Run_against_Drug_Abuse,_on_the_occasion_of_the_International_Day_Against_Drug_Abuse_and_Illicit_Trafficking,_at_India_Gate,_in_New_Delhi_on_June_26,_2011.jpg
Updated Humayun's Tomb with image URL: https://commons.wikimedia.org/wiki/Special:FilePath/File:"Capture_of_the_King_of_Delhi_by_Captain_Hodson".jpg
Updated Akshardham Temple with image URL: https://images.unsplash.com/photo-1671137777317-877064fb882d?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2NzM1Njd8MHwxfHNlYXJjaHwxfHxBa3NoYXJkaGFtJTIwVGVtcGxlfGVufDB8fHx8MTczMTE0Njg5M3ww&ixlib=rb-4.0.3&q=80&w=1080
Updated Waste to Wonder Park with image URL: https://images.unsplash.com/photo-1686052903991-fedc0903ad4d?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2NzM1Njd8MHwxfHNlYXJjaHwxfHxXYXN0ZSUyMHRvJTIwV29uZGVyJTIwUGFya3xlbnwwfHx8fDE3MzExNDY4OTV8MA&ixlib=rb-4.0.3&q=80&w=1080
Updated Jantar Mantar with image URL: https://commons.wikimed

In [22]:
import firebase_admin
from firebase_admin import credentials, firestore

# Check if the Firebase app has already been initialized
if not firebase_admin._apps:
    # Initialize Firebase Admin SDK if not initialized
    cred = credentials.Certificate('C:\\Users\\AKHNA\\Downloads\\smart-tour-guide-3d30d-firebase-adminsdk-h17r2-bda949eabd.json')  # Update with your credentials path
    firebase_admin.initialize_app(cred)

# Initialize Firestore client
db = firestore.client()

# Function to remove the 'image_url' field from Firestore documents
def remove_image_url_field():
    locations_ref = db.collection('locations')

    # Loop through each document in the 'locations' collection
    for doc in locations_ref.stream():
        doc_ref = locations_ref.document(doc.id)
        
        # Remove the 'image_url' field
        doc_ref.update({'image_url': firestore.DELETE_FIELD})
        print(f"Removed 'image_url' field from {doc.id}")

# Run the function to remove the 'image_url' field from all documents
remove_image_url_field()
print("Removed 'image_url' field from all documents in Firestore.")


Removed 'image_url' field from 0
Removed 'image_url' field from 1
Removed 'image_url' field from 10
Removed 'image_url' field from 100
Removed 'image_url' field from 101
Removed 'image_url' field from 102
Removed 'image_url' field from 103
Removed 'image_url' field from 104
Removed 'image_url' field from 105
Removed 'image_url' field from 106
Removed 'image_url' field from 107
Removed 'image_url' field from 108
Removed 'image_url' field from 109
Removed 'image_url' field from 11
Removed 'image_url' field from 110
Removed 'image_url' field from 111
Removed 'image_url' field from 112
Removed 'image_url' field from 113
Removed 'image_url' field from 114
Removed 'image_url' field from 115
Removed 'image_url' field from 116
Removed 'image_url' field from 117
Removed 'image_url' field from 118
Removed 'image_url' field from 119
Removed 'image_url' field from 12
Removed 'image_url' field from 120
Removed 'image_url' field from 121
Removed 'image_url' field from 122
Removed 'image_url' field f